In [115]:
from BitFlipper import add_BitFlips
from Qubit_Simulator import pad_counts
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
import numpy as np
import random
from tqdm import tqdm

def CZ_errors(num_gates):
    qc = QuantumCircuit(2)

    for i in range(num_gates):
        qc.cz(0,1)
    qc.measure_all()

    return qc

In [177]:
# sim_seeds = np.random.randint(0,1073741824,1048576,dtype=int)
sim_seeds = np.load('../data/sim_seeds.npy')
sim_seeds = np.array(sim_seeds,dtype = int)/10000

In [179]:
shots_per_circuit = 32
e_g1 = 0.0035
e_g2 = 0.0027
num_gates = round(0.2/(max(e_g1,e_g2)))
test_circuit = CZ_errors(num_gates)

simulator = AerSimulator()
flipped_counts = np.zeros(4)

for sim_seed in tqdm(sim_seeds):
    flipped_circuit = add_BitFlips(test_circuit, seed = sim_seed, gate_errors = [e_g1,e_g2])
    flipped_result = simulator.run(flipped_circuit, shots=shots_per_circuit).result()
    
    counts = pad_counts(flipped_result.get_counts(),2)
    bitstring_list = sorted(list(set(counts.keys())))
    bitstring_counts = [counts.get(key, 0) for key in bitstring_list]
    flipped_counts += bitstring_counts

flipped_counts
# noisy_counts = pad_counts(noisy_result.get_counts(),num_qubits)

100%|███████████████████████████████████| 1048576/1048576 [45:18<00:00, 385.77it/s]


array([24294816.,  4802304.,  3720576.,   736736.])

In [168]:
print(float(flipped_counts[3]/sum(flipped_counts)))

P_1X = ( 1 - ( ( 1 - 2 * e_g1) ** num_gates ) ) / 2
P_X1 = ( 1 - ( ( 1 - 2 * e_g2) ** num_gates ) ) / 2

a = ((1+e_g1)**num_gates - (1-e_g1)**num_gates)/2 
b = ((1+e_g2)**num_gates - (1-e_g2)**num_gates)/2
print(a*b)

0.021805763244628906
0.03101220112585013


In [20]:
flipped_counts
e_m01 = [0.018,0.012]
e_m10 = [0.0165,0.0145]

full_error_counts = fast_interaction_multiply(flipped_counts,2,e_m01,e_m10)

array([4096.,    0.,    0.,    0.])